The Best Attained Accuracy Model For Proposed Dataset.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

# ====================================================================
# 1. CONFIGURATION
# ====================================================================

# --- Set this to the folder containing your emotion sub-folders ---
DATASET_PATH = "C:/Users/assen/Downloads/CROP/CROP/" # Using the path from your screenshot

# --- Model & Training Parameters ---
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16 # Can be increased slightly with efficient preprocessing
NUM_CLASSES = 6
EPOCHS = 25 # Let's train for a few more epochs as the model might learn more

# ====================================================================
# 2. ADVANCED PREPROCESSING FUNCTION
# ====================================================================

def preprocess_thermal_image(image_path, label):
    """
    This function takes an image file path, loads it, resizes it, applies 
    Grayscale/CLAHE, and prepares it for the model.
    """
    def _apply_preprocessing(path):
        # Decode the path and read the image with OpenCV
        path = path.numpy().decode('utf-8')
        img_array = cv2.imread(path)
        
        # --- FIX: Resize is now done here using cv2 ---
        # This ensures every image is the same size before other processing.
        resized_img = cv2.resize(img_array, IMAGE_SIZE)
        
        # Perform all other operations on the already resized image
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(gray_img)
        final_img = cv2.cvtColor(enhanced_img, cv2.COLOR_GRAY2BGR)
        
        return final_img

    # Apply the OpenCV function using tf.py_function
    image = tf.py_function(_apply_preprocessing, [image_path], tf.uint8)
    
    # Now that the image is always 224x224, we just need to tell TensorFlow
    # to trust us on the shape. This is crucial.
    image.set_shape(IMAGE_SIZE + (3,))
    
    return image, label

# ====================================================================
# 3. LOAD AND PREPARE DATASET
# ====================================================================

print(f"Loading images from: {DATASET_PATH}")
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Directory not found: {DATASET_PATH}")

# Create a dataset of all file paths
full_dataset = tf.data.Dataset.list_files(os.path.join(DATASET_PATH, '*/*'), shuffle=True, seed=123)

# Get class names from subdirectories
class_names = np.array(sorted([item.name for item in os.scandir(DATASET_PATH) if item.is_dir()]))
print("✅ Found classes:", list(class_names))

def get_label(file_path):
    # Extracts the parent directory name (e.g., 'Happy') and finds its index
    parts = tf.strings.split(file_path, os.path.sep)
    return tf.argmax(parts[-2] == class_names)

def process_path(file_path):
    label = get_label(file_path)
    # We pass the file_path itself to our preprocessing function
    return file_path, label

# Create a labeled dataset of (image_path, label_index)
labeled_dataset = full_dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

# --- Split the data into training (80%) and validation (20%) sets ---
DATASET_SIZE = tf.data.experimental.cardinality(labeled_dataset).numpy()
train_size = int(0.8 * DATASET_SIZE)

train_data = labeled_dataset.take(train_size)
validation_data = labeled_dataset.skip(train_size)

# --- Apply the new preprocessing and batch the data ---
# This .map() call is where the magic happens!
print("Applying advanced preprocessing to the dataset...")
train_dataset = train_data.map(preprocess_thermal_image, num_parallel_calls=tf.data.AUTOTUNE)
validation_dataset = validation_data.map(preprocess_thermal_image, num_parallel_calls=tf.data.AUTOTUNE)

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)

print("✅ New preprocessing pipeline is ready!")

# ====================================================================
# 4. BUILD, COMPILE, AND TRAIN THE MODEL (UNCHANGED)
# ====================================================================

# We use the exact same model architecture to ensure a fair comparison
input_shape = IMAGE_SIZE + (3,)
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False

# Create the new model on top
inputs = Input(shape=input_shape)
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs) # Use the built-in preprocessing
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

# Train the model
print("\n--- Starting Training with Advanced Preprocessing ---")
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)

# ====================================================================
# 5. EVALUATE AND VISUALIZE RESULTS
# ====================================================================

print("\n--- Evaluating Final Model ---")
loss, accuracy = model.evaluate(validation_dataset)
print(f"\nFinal Validation Accuracy: {accuracy*100:.2f}%")
print(f"Final Validation Loss: {loss:.4f}")

# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.suptitle('Model Training History with Advanced Preprocessing 📊')
plt.show()

Generating Confusion Matrix...

In [ ]:
# ====================================================================
# 4. GENERATE REPORT AND CONFUSION MATRIX
# ====================================================================

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

print("-" * 60)
print("--- Generating Final Report ---")
print("-" * 60)

# --- Get all predictions from the validation set ---
true_labels = []
predicted_labels = []

for images, labels in validation_dataset:
    true_labels.extend(labels.numpy())
    preds = model.predict(images, verbose=0)
    predicted_labels.extend(np.argmax(preds, axis=1))

# --- Print the detailed Classification Report ---
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names, labels=np.arange(NUM_CLASSES), zero_division=0))

# --- Display the Confusion Matrix ---
print("\nConfusion Matrix:")
cm = confusion_matrix(true_labels, predicted_labels, labels=np.arange(NUM_CLASSES))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.xticks(rotation=45)
plt.show()